## Importing Libraries and Datasets

In [ ]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
import re
import ast
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [ ]:
df=pd.read_csv('crm_activities_100_voc.csv')

In [ ]:
df.rename(columns={
    "POTENTIALID": "deal_id",
    "ACTIVITY_TIMESTAMP": "timestamp",
    "ACTIVITY": "activity_type",
    "CONTACTID": "contact_id",
    "ACCOUNTID": "account_id",
    "CALL_VOC": "call_voc",
    "EMAIL_VOC": "email_voc",
    "EVENT_VOC": "event_voc",
    "INVOICE_VOC": "invoice_voc",
    "QUOTE_VOC": "quote_voc",
    "SALESORDER_VOC": "salesorder_voc",
    "DEALSTAGE": "deal_stage",
    "CLASS_TAG": "class_tag",
    "INVOICEGRANDTOTAL": "deal_amount",
}, inplace=True)

df["timestamp"] = pd.to_datetime(df["timestamp"])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   deal_id         800 non-null    object        
 1   timestamp       800 non-null    datetime64[ns]
 2   activity_type   800 non-null    object        
 3   contact_id      800 non-null    object        
 4   account_id      800 non-null    object        
 5   deal_amount     100 non-null    float64       
 6   deal_stage      800 non-null    object        
 7   email_voc       120 non-null    object        
 8   call_voc        260 non-null    object        
 9   event_voc       120 non-null    object        
 10  quote_voc       100 non-null    object        
 11  salesorder_voc  100 non-null    object        
 12  invoice_voc     100 non-null    object        
 13  class_tag       800 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(12)
memory usage:

## Extracting and mapping VOC columns


In [ ]:
# Mapping activity_type to corresponding voc column
voc_mapping = {
    'mail': 'email_voc',
    'call': 'call_voc',
    'meeting': 'event_voc',
    'quote': 'quote_voc',
    'salesorder': 'salesorder_voc',
    'invoice': 'invoice_voc'
}

# Function to get voc dict based on activity_type
def extract_voc(row):
    voc_col = voc_mapping.get(row['activity_type'])
    voc_data = row[voc_col]
    if pd.isna(voc_data):
        return None
    if isinstance(voc_data, str):
        try:
            voc_data = ast.literal_eval(voc_data)  # convert string dict to actual dict
        except Exception:
            return None
    return voc_data if isinstance(voc_data, dict) else None

# Extract into separate columns
df['voc_dict'] = df.apply(extract_voc, axis=1)
df['sentiment'] = df['voc_dict'].apply(lambda x: x.get('sentiment') if isinstance(x, dict) else None)
df['intent'] = df['voc_dict'].apply(lambda x: x.get('intent') if isinstance(x, dict) else None)
df['emotion'] = df['voc_dict'].apply(lambda x: x.get('emotion') if isinstance(x, dict) else None)

# Drop temp column if not needed
df.drop(columns=['voc_dict'], inplace=True)

sentiment_map = {"positive": 1, "neutral": 0.5, "negative": 0.2}
intent_map = {
    "purchase": 1.0, "gratitude": 0.9, "request": 0.8, "query": 0.6,
    "complaint": 0.2, "others": 0.3
}
emotion_map = {
    "happy": 1.0, "gratitude": 0.9, "trust": 0.9,
    "neutral": 0.5, "confusion": 0.3, "frustration": 0.2, "discontentment": 0.2
}

df["sentiment_score"] = df["sentiment"].map(sentiment_map)
df["intent_score"] = df["intent"].map(intent_map)
df["emotion_score"] = df["emotion"].map(emotion_map)

## Handling missing values

In [ ]:
df['intent_score'] = df['intent_score'].fillna(df['intent_score'].mean())
df['sentiment_score'] = df['sentiment_score'].fillna(df['sentiment_score'].mean())
df['emotion_score'] = df['emotion_score'].fillna(df['emotion_score'].mean())

## Helper Functions

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
def interpret_trend(scores, feature_name="", min_net_change=0.05, min_speed=0.01):
    clean = [s for s in scores if s is not None and not np.isnan(s)]
    n = len(clean)
    if n < 2:
        return {"label": "Insufficient Data", "n": n}

    x = np.arange(n)
    slope = np.polyfit(x, clean, 1)[0]
    net_change = slope * (n - 1)

    direction = "Improving" if net_change > 0 else "Worsening" if net_change < 0 else "Stable"

    if abs(net_change) < min_net_change:
        label = "Stable"
    elif abs(slope) >= min_speed:
        label = f"{direction} fast"
    else:
        label = f"{direction} slowly"

    return {
        "label": label,
        "slope": round(slope, 5),
        "net_change": round(net_change, 3),
        "n": n
    }

In [ ]:
def combine_trends(sentiment_label, intent_label, emotion_label):
    """Combine 3 labels into a single overall label."""
    trends = [sentiment_label, intent_label, emotion_label]

    improving_count = sum("Improving" in t for t in trends)
    worsening_count = sum("Worsening" in t for t in trends)

    if improving_count >= 2:
        return "Overall Improving"
    elif worsening_count >= 2:
        return "Overall Worsening"
    else:
        return "Overall Stable"

In [ ]:
def label_deal_info(label, deal_duration_days, class_tag):
    """
    Modifies the label based on actual deal outcome and time taken.
    """
    outcome = class_tag.lower()

    trend_label = label

    # Adjust for outcome
    if "improving" in label.lower() and outcome == "closed lost":
        trend_label += f" but Lost"
    elif "improving" in label.lower() and outcome == "closed won":
        trend_label += f" and Won"
    elif "worsening" in label.lower() and outcome == "closed won":
        trend_label += f" but Won"
    elif "worsening" in label.lower() and outcome == "closed lost":
        trend_label += f" and Lost"
    else:
        result = "Won" if outcome == "closed won" else "Lost"
        trend_label += f" but {result}"

    # Adjust for deal speed
    if deal_duration_days < 10:
        trend_label += f" - Quick Cycle(<10days)"
    elif deal_duration_days > 60:
        trend_label += f" - Long Cycle(>60days)"
    else:
        trend_label += f" - Normal Cycle(20-60days)"

    return trend_label

## Labelling existing Deals

In [ ]:
# Group by deal and run for each score type
deal_trends = []
for deal_id, group in df.groupby('deal_id'):
    group = group.sort_values('timestamp')
    deal_duration_days = (group['timestamp'].max() - group['timestamp'].min()).days
    class_tag = group['class_tag'].iloc[-1]
    class_tag_encoded = group['class_tag'].apply(lambda x: 1 if x == "closed won" else 0).iloc[-1]

    # Calculate trends
    intent_result = interpret_trend(group['intent_score'].tolist(), "intent")
    sentiment_result = interpret_trend(group['sentiment_score'].tolist(), "sentiment")
    emotion_result = interpret_trend(group['emotion_score'].tolist(), "emotion")

    # Combine into one label
    overall_label = combine_trends(sentiment_result['label'], intent_result['label'], emotion_result['label'])

    # Apply deal context
    overall_label = label_deal_info(overall_label, deal_duration_days, class_tag)

    deal_trends.append({
        "deal_id": deal_id,
        'deal_duration_days': deal_duration_days,
        'class_tag': class_tag,
        "intent_label": intent_result['label'],
        "sentiment_label": sentiment_result['label'],
        "emotion_label": emotion_result['label'],
        "deal_trend": overall_label,
        "intent_slope": intent_result['slope'],
        "sentiment_slope": sentiment_result['slope'],
        "emotion_slope": emotion_result['slope'],
    })

deal_trend_df = pd.DataFrame(deal_trends)
deal_trend_df

,deal_id,deal_duration_days,class_tag,intent_label,sentiment_label,emotion_label,deal_trend,intent_slope,sentiment_slope,emotion_slope
0,01421d86-5916-4993-b3da-4997d981549f,0,closed won,Improving fast,Improving fast,Improving fast,Overall Improving and Won - Quick Cycle(<10days),0.06667,0.07143,0.07619
1,03aa496b-6ba7-4091-839c-a8a1f91c3b34,0,closed won,Improving fast,Improving fast,Improving fast,Overall Improving and Won - Quick Cycle(<10days),0.09167,0.05357,0.06848
2,0a669394-eb58-4b46-931b-9c776b2087da,0,closed won,Improving fast,Improving fast,Improving fast,Overall Improving and Won - Quick Cycle(<10days),0.08571,0.08929,0.09048
3,15117095-d2c8-4881-8a66-4e958918a9fa,0,closed won,Improving fast,Improving fast,Improving fast,Overall Improving and Won - Quick Cycle(<10days),0.04762,0.01786,0.02619
4,16824ec1-7d99-469e-80f3-a270206458ce,0,closed won,Improving fast,Improving fast,Improving fast,Overall Improving and Won - Quick Cycle(<10days),0.06640,0.04863,0.04944
...,...,...,...,...,...,...,...,...,...,...
115,f94a1f33-ab63-400b-80c6-4f05ece9f7b1,0,closed won,Improving fast,Improving fast,Improving fast,Overall Improving and Won - Quick Cycle(<10days),0.06429,0.02976,0.03810
116,f9c01ca9-e380-41e4-b626-c0023dbbb921,0,closed lost,Improving fast,Stable,Stable,Overall Stable but Lost - Quick Cycle(<10days),0.04000,-0.00000,0.01000
117,fc15ef3d-6956-4c77-9295-f4c0b3883438,0,closed won,Improving fast,Stable,Improving fast,Overall Improving and Won - Quick Cycle(<10days),0.04571,0.00000,0.02286
118,fc1f3940-1a42-4a51-ba97-3bbb02c497e5,0,closed won,Improving fast,Improving fast,Improving fast,Overall Improving and Won - Quick Cycle(<10days),0.09429,0.07143,0.08286


## Deal Label Prediction

In [ ]:
def label_new_deal(deal_id, contact_records, deal_records, activity_records):
    df_contacts = pd.DataFrame(contact_records)
    df_deals = pd.DataFrame(deal_records)
    df_activities = pd.DataFrame(activity_records)

    # Merge contacts and activities first
    merged = pd.merge(df_activities, df_deals, on="deal_id")
    merged = pd.merge(merged, df_contacts, on="contact_id")

    # Filter for the given account_id
    merged = merged[merged["deal_id"] == deal_id]

    display(merged)

    # Map scores
    merged["sentiment_score"] = merged["sentiment"].map(sentiment_map)
    merged["intent_score"] = merged["intent"].map(intent_map)
    merged["emotion_score"] = merged["emotion"].map(emotion_map)

    merged = merged.sort_values('timestamp')
    deal_duration_days = (merged['timestamp'].max() - merged['timestamp'].min()).days
    print(deal_duration_days)
    class_tag = merged['class_tag'].iloc[-1]
    class_tag_encoded = merged['class_tag'].apply(lambda x: 1 if x == "closed won" else 0).iloc[-1]

    # Calculate trends
    intent_result = interpret_trend(merged['intent_score'].tolist(), "intent")
    sentiment_result = interpret_trend(merged['sentiment_score'].tolist(), "sentiment")
    emotion_result = interpret_trend(merged['emotion_score'].tolist(), "emotion")

    # Combine into one label
    overall_label = combine_trends(sentiment_result['label'], intent_result['label'], emotion_result['label'])

    # Apply deal context
    overall_label = label_deal_info(overall_label, deal_duration_days, class_tag)

    return {
        "deal_id": deal_id,
        'deal_duration_days': deal_duration_days,
        'class_tag': class_tag,
        "intent_label": intent_result['label'],
        "sentiment_label": sentiment_result['label'],
        "emotion_label": emotion_result['label'],
        "deal_trend": overall_label,
        "intent_slope": intent_result['slope'],
        "sentiment_slope": sentiment_result['slope'],
        "emotion_slope": emotion_result['slope'],
    }

In [ ]:
example_deal_records = [
    {"deal_id": "DX1", "contact_id": "C1", "deal_amount": 9000, "deal_stage": "Proposal", "class_tag": "closed lost"},
    {"deal_id": "DX2", "contact_id": "C1", "deal_amount": 14000, "deal_stage": "Won", "class_tag": "closed won"},
]

example_contact_records = [
    {"contact_id": "C1", "account_id": "AC101"},
]

example_activity_records = [
    {
        "deal_id": "DX1",
        "activity_type": "call",
        "timestamp": datetime.now() - timedelta(days=5),
        "intent": "complaint",
        "emotion": "frustration",
        "sentiment": "negative"
    },
    {
        "deal_id": "DX1",
        "activity_type": "call",
        "timestamp": datetime.now() - timedelta(days=3),
        "intent": "complaint",
        "emotion": "frustration",
        "sentiment": "negative"
    },
    {
        "deal_id": "DX1",
        "activity_type": "mail",
        "timestamp": datetime.now() - timedelta(days=7),
        "intent": "complaint",
        "emotion": "frustration",
        "sentiment": "neutral"
    },
    {
        "deal_id": "DX1",
        "activity_type": "meeting",
        "timestamp": datetime.now() - timedelta(days=8),
        "intent": "purchase",
        "emotion": "happy",
        "sentiment": "positive"
    },
    {
        "deal_id": "DX1",
        "activity_type": "call",
        "timestamp": datetime.now() - timedelta(days=10),
        "intent": "purchase",
        "emotion": "happy",
        "sentiment": "positive"
    },
    {
        "deal_id": "DX2",
        "activity_type": "quote",
        "timestamp": datetime.now() - timedelta(days=5),
        "intent": "purchase",
        "emotion": "happy",
        "sentiment": "positive"
    },
    {
        "deal_id": "DX2",
        "activity_type": "invoice",
        "timestamp": datetime.now() - timedelta(days=2),
        "intent": "purchase",
        "emotion": "happy",
        "sentiment": "positive"
    },
    {
        "deal_id": "DX2",
        "activity_type": "call",
        "timestamp": datetime.now() - timedelta(days=15),
        "intent": "complaint",
        "emotion": "frustration",
        "sentiment": "negative"
    },
    {
        "deal_id": "DX2",
        "activity_type": "meeting",
        "timestamp": datetime.now() - timedelta(days=20),
        "intent": "complaint",
        "emotion": "frustration",
        "sentiment": "negative"
    },
    {
        "deal_id": "DX2",
        "activity_type": "mail",
        "timestamp": datetime.now() - timedelta(days=25),
        "intent": "complaint",
        "emotion": "frustration",
        "sentiment": "negative"
    },
]

In [ ]:
new_deal_label = label_new_deal("DX2", example_contact_records, example_deal_records, example_activity_records)
print(json.dumps(new_deal_label, indent=2))

,deal_id,activity_type,timestamp,intent,emotion,sentiment,contact_id,deal_amount,deal_stage,class_tag,account_id
5,DX2,quote,2025-08-12 14:48:18.969601,purchase,happy,positive,C1,14000,Won,closed won,AC101
6,DX2,invoice,2025-08-15 14:48:18.969602,purchase,happy,positive,C1,14000,Won,closed won,AC101
7,DX2,call,2025-08-02 14:48:18.969603,complaint,frustration,negative,C1,14000,Won,closed won,AC101
8,DX2,meeting,2025-07-28 14:48:18.969604,complaint,frustration,negative,C1,14000,Won,closed won,AC101
9,DX2,mail,2025-07-23 14:48:18.969605,complaint,frustration,negative,C1,14000,Won,closed won,AC101


22
{
  "deal_id": "DX2",
  "deal_duration_days": 22,
  "class_tag": "closed won",
  "intent_label": "Improving fast",
  "sentiment_label": "Improving fast",
  "emotion_label": "Improving fast",
  "deal_trend": "Overall Improving and Won - Normal Cycle(20-60days)",
  "intent_slope": 0.24,
  "sentiment_slope": 0.24,
  "emotion_slope": 0.24
}


In [ ]:
new_deal_label = label_new_deal("DX1", example_contact_records, example_deal_records, example_activity_records)
print(json.dumps(new_deal_label, indent=2))

,deal_id,activity_type,timestamp,intent,emotion,sentiment,contact_id,deal_amount,deal_stage,class_tag,account_id
0,DX1,call,2025-08-12 14:48:18.969588,complaint,frustration,negative,C1,9000,Proposal,closed lost,AC101
1,DX1,call,2025-08-14 14:48:18.969595,complaint,frustration,negative,C1,9000,Proposal,closed lost,AC101
2,DX1,mail,2025-08-10 14:48:18.969597,complaint,frustration,neutral,C1,9000,Proposal,closed lost,AC101
3,DX1,meeting,2025-08-09 14:48:18.969598,purchase,happy,positive,C1,9000,Proposal,closed lost,AC101
4,DX1,call,2025-08-07 14:48:18.969599,purchase,happy,positive,C1,9000,Proposal,closed lost,AC101


6
{
  "deal_id": "DX1",
  "deal_duration_days": 6,
  "class_tag": "closed lost",
  "intent_label": "Worsening fast",
  "sentiment_label": "Worsening fast",
  "emotion_label": "Worsening fast",
  "deal_trend": "Overall Worsening and Lost - Quick Cycle(<10days)",
  "intent_slope": -0.24,
  "sentiment_slope": -0.24,
  "emotion_slope": -0.24
}
